In [2]:
import numpy as np
import pandas as pd
import mlflow
import joblib

In [17]:
def load_model(run_id: str = None):
    
    if run_id is None:
        model_path = 'out/model.sav'
    else:
        mlflow_client = mlflow.tracking.MlflowClient()
        model_path = mlflow_client.download_artifacts(run_id, 'model.sav')
        
    return joblib.load(model_path)

In [15]:
def predict(ds: pd.DataFrame, model) -> pd.DataFrame:
    x = ds.iloc[:, 1:]
    y_log1p = model.predict(x)
    y = np.expm1(y_log1p)
    return pd.DataFrame({
        'row_id': ds['row_id'],
        'meter_reading': y,
    })

In [12]:
model = load_model('a04bd1e7c0c74a78aae6607882acffc8')
dataset_test = pd.read_csv('dataset_test.csv').sample(frac=0.01)

In [16]:
predict(dataset_test, model)

,row_id,meter_reading
144202,144202,111.187358
917342,917342,1419.220513
1443966,1443966,0.315463
2240445,2240445,884.425638
2858716,2858716,29.311737
...,...,...
2411286,2411286,0.939451
4669175,4669175,509.535846
3581213,3581213,78.535834
735593,735593,6134.289949
